# Now updated for the test

## Libraries

In [1]:
import numpy as np
import pandas as pd
import xlrd
import os

In [2]:
import sys
np.set_printoptions(threshold=np.inf)

## Mount Google Drive

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
#os.listdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
os.listdir(".")

In [ ]:
housePath = "/content/drive/My Drive/Colab Notebooks/house_prices.xlsx"

# Raw Population to dataframe

In [ ]:
# xl3 = pd.ExcelFile("/content/drive/My Drive/Colab Notebooks/idaho_county_population_2000_to_2010.xlsx")
# xl4 = pd.ExcelFile("/content/drive/My Drive/Colab Notebooks/idaho_county_population_2010_to_2018.xlsx")
xl3 = pd.ExcelFile("./Data/idaho_county_population_2000_to_2010.xlsx")
xl4 = pd.ExcelFile("./Data/idaho_county_population_2010_to_2018.xlsx")

In [ ]:
dfpop1 = xl3.parse("main")
dfpop1.to_pickle('rawPop1.pkl')
dfpop2 = xl4.parse("main")
dfpop2.to_pickle('rawPop2.pkl')

### This code to clean and prep population 00-10 from pickle

In [ ]:
rawPop1 = pd.read_pickle('rawPop1.pkl')
temp1 = rawPop1.copy()
temp1 = temp1.iloc[2:48]
temp1 = temp1.replace(['County','\.','\s'],'', regex = True)
cTemp = temp1['table with row headers in column A and column headers in rows 3 through 4 (leading dots indicate sub-parts)']
temp1 = temp1.drop(['table with row headers in column A and column headers in rows 3 through 4 (leading dots indicate sub-parts)',
                    'Unnamed: 1','Unnamed: 12','Unnamed: 13'], axis = 1)
temp1 = temp1.astype(int)
temp1.columns = temp1.iloc[0]
temp1 = temp1.iloc[2:]
temp1['County'] = cTemp
pop1 = temp1.copy()
pop1.head(3)


### This code to clean and prep population 10-18 from pickle

In [ ]:
rawPop2 = pd.read_pickle('rawPop2.pkl')
temp2 = rawPop2.copy()
temp2 = temp2.replace(['Population Estimate \(as of July 1\) \- ','County, Idaho'],'', regex = True)
temp2.columns = temp2.iloc[0]
#temp2 = temp_df.iloc[2:]
cT2 = temp2['Geography']
temp2 = temp2.drop(['Id','Id2','Geography','April 1, 2010 - Census',
                     'April 1, 2010 - Estimates Base'], axis = 1)
temp2 = temp2.astype(int)
temp2['County'] = cT2
temp2 = temp2.iloc[1:]
temp2 = temp2.replace('\s','', regex = True)

pop2 = temp2.copy()
pop2.head(3)

### Merge the cleaned data frames for the 00-09 pop and 10-18 pop and pickle the result (totalPop)

In [ ]:
totalPop = pd.merge(pop2,pop1, on = 'County')

# Reorder columns, not all column titles are str some are int, going to 
# transpose df so will turn them all into factors then


totalPop = totalPop[["County",2000,2001,2002,2003,2004,2005,2006,2007,
                    2008,2009,"2010","2011","2012","2013","2014","2015",
                    "2016","2017","2018"]]


totalPop.to_pickle('totalPop.pkl')
totalPop.head(5)

### From totalPop.pkl to metroPop, then pickle

In [ ]:
totalPop = pd.read_pickle('totalPop.pkl')
totalPop = totalPop.iloc[[0,7,13,22,36],:]

temp3 = totalPop.copy()
temp3 = temp3.T
temp3.columns = temp3.iloc[0]
temp3 = temp3.iloc[1:]
temp3 = temp3.reset_index()
temp3 = temp3.rename(columns={'index':'Year'})

# Add the columns together for a total population
temp3['TotalPop'] = temp3['Ada']+temp3['Boise']+temp3['Canyon']+temp3['Gem']+temp3['Owyhee']
metroPop = temp3.copy()
pop_year_totals = metroPop.drop(columns=['Ada','Boise','Canyon','Gem','Owyhee'])

metroPop = metroPop.astype('int')

metroPop['Year'] = metroPop['Year'].astype(str)
metroPop['Year'] = metroPop['Year'].astype('datetime64[Y]')

metroPop.to_pickle('metroPop.pkl')
metroPop.head(5)

# House Count

In [ ]:
## Load House excel files and pickle

xl5 = pd.ExcelFile("./Data/idaho_housing_unit_count_2000_to_2010.xlsx")
xl6 = pd.ExcelFile("./Data/idaho_housing_unit_count_2010_to_2018.xlsx")

dfH1 = xl5.parse("main")
dfH1.to_pickle('rawHouse1.pkl') 
dfH2 = xl6.parse("main")
dfH2.to_pickle('rawHouse2.pkl')
#dfH1.head(5)

In [ ]:
### From rawHouse1.pkl clean

temp5 = pd.read_pickle('rawHouse1.pkl')
temp5 = temp5.iloc[2:48]
# temp1 = temp1.replace(['County','\.','\s'],'', regex = True)
cTemp5 = temp5['table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)']
temp5 = temp5.drop(['table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)',
                    'Unnamed: 1','Unnamed: 12','Unnamed: 13'], axis = 1)
temp5 = temp5.astype(int)
temp5.columns = temp5.iloc[0]
temp5 = temp5.iloc[2:]
temp5['County'] = cTemp

house1 = temp5.copy()
house1.head(3)

In [ ]:
### From rawHouse2.pkl clean

temp6 = pd.read_pickle('rawHouse2.pkl')
temp6 = temp6.replace(['Housing Unit Estimate \(as of July 1\) \- ','County, Idaho'],'', regex = True)
temp6.columns = temp6.iloc[0]

cT6 = temp6['Geography']
temp6 = temp6.drop(['Id','Id2','Geography','April 1, 2010 - Census',
                     'April 1, 2010 - Estimates Base'], axis = 1)
temp6 = temp6.astype(int)
temp6['County'] = cT6
temp6 = temp6.iloc[1:]
temp6 = temp6.replace('\s','', regex = True)

house2 = temp6.copy()
house2.head(3)

### Combine cleaned dataframes to merge into totalHouse and pickle

In [ ]:
totalHouse = pd.merge(house2,house1, on = 'County')

# Reorder columns, not all column titles are str some are int, going to 
# transpose df so will turn them all into factors then
totalHouse = totalHouse[["County",2000,2001,2002,2003,2004,2005,2006,2007,
                    2008,2009,"2010","2011","2012","2013","2014","2015",
                    "2016","2017","2018"]]

totalHouse.to_pickle('totalHouse.pkl')
totalHouse.head(3)

### From totalHouse.pkl make metroHouse and pickle it

In [ ]:
metroHouseT = pd.read_pickle('totalHouse.pkl')
metroHouseT = metroHouseT.iloc[[0,7,13,22,36],:]

tempH = metroHouseT.copy()
tempH = tempH.T
tempH.columns = tempH.iloc[0]
tempH = tempH.iloc[1:]
tempH = tempH.reset_index()
tempH = tempH.rename(columns={'index':'Year'})

# Add columns together for a total house count
tempH['TotalHouse'] = tempH['Ada']+tempH['Boise']+tempH['Canyon']+tempH['Gem']+tempH['Owyhee']

metroHouse = tempH.copy()
metroHouse = metroHouse.astype('int')

metroHouse['Year'] = metroHouse['Year'].astype(str)
metroHouse['Year'] = metroHouse['Year'].astype('datetime64[Y]')

metroHouse.to_pickle('metroHouse.pkl')
metroHouse.head(3)

In [ ]:
metroHouse = pd.read_pickle('metroHouse.pkl')
metroHouse.dtypes

## Create yearTotals from metroHouse.pkl, metroPop.pkl, and job

In [ ]:
metroHouse = pd.read_pickle('metroHouse.pkl')
metroPop = pd.read_pickle('metroPop.pkl')
metroJobs = pd.read_pickle('jobTypes.pkl')


yearTotals = metroHouse.copy()
yearTotals = yearTotals.drop(['Ada','Boise','Gem','Canyon','Owyhee'],axis =1)
yearTotals['TotalPop'] = metroPop['TotalPop']
yearTotals['TotalEmp'] = metroJobs['All Occupations']
yearTotals.to_pickle('yearTotals.pkl')
yearTotals

# Year Totals % change in values

In [6]:
df = pd.read_pickle('yearTotals.pkl')

df['Total_House_Change'] = df['TotalHouse'].pct_change()
df['Total_Pop_Change'] = df['TotalPop'].pct_change()
df['Total_Emp_Change'] = df['TotalEmp'].pct_change()

df = df[['Year','Total_House_Change','Total_Pop_Change','Total_Emp_Change']]

df.to_pickle('yrTotalChange.pkl')
df

County,Year,Total_House_Change,Total_Pop_Change,Total_Emp_Change
0,2000,NaN,NaN,NaN
1,2001,0.034815,0.036376,-0.033423
2,2002,0.035035,0.028629,0.016749
3,2003,0.034937,0.024994,0.011204
4,2004,0.037588,0.026311,0.020145
5,2005,0.041150,0.039935,0.091779
6,2006,0.050832,0.043151,0.056108
7,2007,0.034190,0.034608,0.057930
8,2008,0.018916,0.021832,0.006757
9,2009,0.010229,0.013466,-0.077146
